In [1]:
# 1. Core PyTorch first
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118  

# 2. Utility libs
!pip install matplotlib timm

# 3. Segment Anything
!pip install git+https://github.com/facebookresearch/segment-anything.git


# 4. CLIP
!pip install ftfy regex

# 5. OpenCV (with contrib modules for tracker)
!pip install opencv-contrib-python



Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\hello\appdata\local\temp\pip-req-build-kluckw_u
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-kluckw_u'


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import numpy as np
import cv2
import torch
import timm
from segment_anything import sam_model_registry, SamPredictor
import timm
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image
import clip

In [3]:
!curl.exe -L "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth" -o "sam_vit_b.pth"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  357M    0 2539k    0     0  1574k      0  0:03:52  0:00:01  0:03:51 1581k
  1  357M    1 4795k    0     0  2082k      0  0:02:55  0:00:02  0:02:53 2087k
  1  357M    1 6523k    0     0  1858k      0  0:03:17  0:00:03  0:03:14 1861k
  2  357M    2 7387k    0     0  1711k      0  0:03:33  0:00:04  0:03:29 1714k
  2  357M    2 8619k    0     0  1618k      0  0:03:46  0:00:05  0:03:41 1723k
  3  357M    3 13.4M    0     0  2193k      0  0:02:46  0:00:06  0:02:40 2406k
  5  357M    5 18.5M    0     0  2601k      0  0:02:20  0:00:07  0:02:13 2840k
  6  357M    6 24.0M    0     0  2970k      0  0:02:03  0:00:08  0:01:55 3785k
  8  357M    8 29.1M    0     0  3205k      0  0:01

In [4]:
import os
print(os.path.exists("sam_vit_b.pth"))  # Should be True


True


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load model (point to the checkpoint you downloaded)
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth")
sam.to(device)
predictor = SamPredictor(sam)

In [6]:
# --------------------------
# 2. CLIP (Option 3: re-implement + load weights)
# --------------------------
!pip install git+https://github.com/openai/CLIP.git
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

# import open_clip
# clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
#     'ViT-B-32', pretrained='openai'
# )
# clip_model = clip_model.eval().cuda()

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hello\AppData\Local\Temp\pip-req-build-0oqw64dw'


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to c:\users\hello\appdata\local\temp\pip-req-build-0oqw64dw
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


In [7]:
clip_model = clip_model.eval().to(device)

In [8]:
# --------------------------
# 3. DINO (Option 3: re-implement + load weights)
# --------------------------
import timm
dino_model = timm.create_model('vit_base_patch16_224', pretrained=True)
dino_model.eval().to(device)

# Preprocess for DINO
dino_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [9]:
# --------------------------
# Helper functions
# --------------------------

def get_sam_mask(image, click_point):
    predictor.set_image(image)
    masks, _, _ = predictor.predict(
        point_coords=np.array([click_point]),
        point_labels=np.array([1]),
        multimask_output=False
    )
    return masks[0]

def get_clip_score(segment_crop, text_query):
    img_proc = clip_preprocess(segment_crop).unsqueeze(0).cuda()
    text = open_clip.tokenize([text_query]).cuda()
    with torch.no_grad():
        img_feat = clip_model.encode_image(img_proc)
        txt_feat = clip_model.encode_text(text)
        img_feat /= img_feat.norm(dim=-1, keepdim=True)
        txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
        score = (img_feat @ txt_feat.T).item()
    return score

def get_dino_score(segment_crop, ref_image):
    img1 = dino_transform(segment_crop).unsqueeze(0).cuda()
    img2 = dino_transform(ref_image).unsqueeze(0).cuda()
    with torch.no_grad():
        f1 = dino_model(img1)
        f2 = dino_model(img2)
        f1 /= f1.norm(dim=-1, keepdim=True)
        f2 /= f2.norm(dim=-1, keepdim=True)
        score = (f1 @ f2.T).item()
    return score

In [21]:
!pip install -U ipywidgets
import ipywidgets as widgets


Defaulting to user installation because normal site-packages is not writeable
  Using cached comm-0.2.3-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   ----------- --------------------------- 41.0/139.8 kB 960.0 kB/s eta 0:00:01
   ------------------- ------------------- 71.7/139.8 kB 653.6 kB/s eta 0:00:01
   --------------------------------- ---- 122.9/139.8 kB 399.4 kB/s eta 0:00:01
   ------------------------------------ - 133.1/139.8 kB 357.2 kB/s eta 0:00:01
   -------------------------------------- 139.8/139.8 kB 307.0 kB/s eta 0:00:00
Using cached comm-0.2.3-py3-none-any.whl (7.3 kB)
   ---------------------------------------- 0.0/216.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/216.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/216.6 kB 217.9 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/216.6 kB 217.9 kB/s eta 0:00:01
   ----- ----------------------------

In [ ]:
# --------------------------
# Interactive Part
# --------------------------

cap = cv2.VideoCapture("C:\\Users\\Hello\\Downloads\\example_videos_car_following.avi") 
ret, frame = cap.read()
cap.release()

image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
Image.fromarray(image_rgb).show()  # display first frame

# ---- Choose mode ----
# ---- Choose mode ----
mode = input("Choose mode (Click / Text / Reference): ")

def run_segmentation(choice):
    if choice.lower() == 'click':
        x = int(input("Enter X coordinate: "))
        y = int(input("Enter Y coordinate: "))
        mask = get_sam_mask(image_rgb, (x,y))
        crop = Image.fromarray(image_rgb[y:y+mask.shape[0], x:x+mask.shape[1]])
        crop.show()
        print("Segmented object from click point.")

    elif choice.lower() == 'text':
        text_query = input("Enter text query: ")
        # Take full image mask first (dummy click in center)
        h, w, _ = image_rgb.shape
        mask = get_sam_mask(image_rgb, (w//2, h//2))
        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        crop = Image.fromarray(image_rgb[y:y+h, x:x+w])
        crop.show()
        score = get_clip_score(crop, text_query)
        print(f"CLIP score for '{text_query}': {score:.4f}")

    elif choice.lower() == 'reference':
        ref_path = input("Enter path to reference image: ")
        ref_image = Image.open(ref_path).convert("RGB")
        h, w, _ = image_rgb.shape
        mask = get_sam_mask(image_rgb, (w//2, h//2))
        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        crop = Image.fromarray(image_rgb[y:y+h, x:x+w])
        crop.show()
        score = get_dino_score(crop, ref_image)
        print(f"DINO score vs reference: {score:.4f}")

    else:
        print("Invalid mode. Choose Click, Text, or Reference.")

# Run directly
run_segmentation(mode)


def run_segmentation(choice):
    if choice == 'Click':
        x = int(input("Enter X coordinate: "))
        y = int(input("Enter Y coordinate: "))
        mask = get_sam_mask(image_rgb, (x,y))
        crop = Image.fromarray(image_rgb[y:y+mask.shape[0], x:x+mask.shape[1]])
        display(crop)
        print("Segmented object from click point.")

    elif choice == 'Text':
        text_query = input("Enter text query: ")
        # Take full image mask first (dummy click in center)
        h,w,_ = image_rgb.shape
        mask = get_sam_mask(image_rgb, (w//2,h//2))
        x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
        crop = Image.fromarray(image_rgb[y:y+h, x:x+w])
        score = get_clip_score(crop, text_query)
        display(crop)
        print(f"CLIP score for '{text_query}': {score:.4f}")

    elif choice == 'Reference':
        uploader = widgets.FileUpload(accept='image/*', multiple=False)
        display(uploader)
        print("Upload a reference image, then rerun scoring cell.")



Choose mode (Click / Text / Reference):  Click
